In [1]:
import pandas as pd
import re
import numpy as np
import json

In [3]:
#Prepare content 

#Gets a list of just the content from (urls against content)
def strip_url_from_content(content):
    return pd.DataFrame(content.items())[1].tolist()

def url_against_page_no(content):
    all_urls_page_num = {
    'page_number': list(range(1, len(content)+1, 1)),
    'url': pd.DataFrame(content.items())[0].tolist()
    }
    return pd.DataFrame(all_urls_page_num)
    

In [4]:
#Splitting out content:
#Split content into each page (all content per page):

def split_by_whole_page(content):
    content = strip_url_from_content(content)
    page_content = pd.DataFrame([content, list(range(1, len(content)+1))]).T
    page_content.columns = ['phrase', 'page_number']
    return page_content
                         

#Split content into sentences per page:
def split_by_sentence(content):
    all_phrases = []
    for page_num, (url, content) in enumerate(content.items(), start=1):
        content = content.split(". ")
        for phrase in content:
            all_phrases.append((phrase, page_num))
    df_phrases = pd.DataFrame(all_phrases, columns = ['phrase', 'page_number'])
    return df_phrases

#Split content into sentences per page:
def split_by_sentence_1(content):
    all_phrases = [
        (phrase, page_num)
        for content, page_num in enumerate(content, start=1)
        for phrase in content.split(". ")
    ]
    df_phrases = pd.DataFrame(all_phrases, columns = ['phrase', 'page_number'])
    return df_phrases



In [5]:
#Define classification (in, not in, exclusion):

#Terms present:
def find_terms_in(terms_list, df_o, terms_list_name):
    df = df_o.copy()
    phrases = df['phrase'].tolist()
    all_terms = []
    for phrase in phrases:
        terms_per_phrase = []
        for term in terms_list:
            if term in phrase:
                present = term
            else: 
                present = 'NaN'
            terms_per_phrase.append(present)
        all_terms.append(terms_per_phrase)
    all_terms = pd.DataFrame(all_terms)
    df[terms_list_name]=all_terms.apply(lambda row: ', '.join(row.values.astype(str)), axis = 1)
    df[terms_list_name] = df[terms_list_name].str.replace('NaN, ', '').str.replace(', NaN', '').str.replace('NaN', '')
    return df

#Find terms not present:
def find_terms_notin(terms_list, df_o, terms_list_name):
    df = df_o.copy()
    phrases = df['phrase'].tolist()
    all_terms = []
    for phrase in phrases:
        terms_per_phrase = []
        for term in terms_list:
            if term not in phrase:
                present = term
            else: 
                present = 'NaN'
            terms_per_phrase.append(present)
        all_terms.append(terms_per_phrase)
    all_terms = pd.DataFrame(all_terms)
    df[terms_list_name]=all_terms.apply(lambda row: ', '.join(row.values.astype(str)), axis = 1)
    df[terms_list_name] = df[terms_list_name].str.replace('NaN, ', '').str.replace(', NaN', '').str.replace('NaN', '')
    return df